In [1]:
# attack the SPEECHCOMMAND models

In [2]:
%load_ext autoreload
%autoreload 2
# Disable jedi autocompleter
%config Completer.use_jedi = False

In [3]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = (8.0, 6.0)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
# check gpus
!gpustat

crescent                Wed Sep 15 12:34:54 2021  418.152.00
[0] GeForce RTX 2080 Ti | 40'C,   0 % |  3621 / 10989 MB | vietanh(2523M) vietanh(1087M)
[1] GeForce RTX 2080 Ti | 40'C,   0 % |   166 / 10989 MB | vietanh(155M)


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# manually set cuda device
torch.cuda.set_device(1)
# device = 'cpu'
print(device)

cuda


/home/felix/.local/lib/python3.6/site-packages/torch/cuda/__init__.py:125: UserWarning: 
GeForce RTX 2080 Ti with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the GeForce RTX 2080 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [7]:
import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [8]:
# load dataset
batch_size = 32
batchsize_for_val = 500
(train_loader, val_loader, test_loader) = fgnh.SpeechCommands_Dataloaders(resample_rate = 8000,
                                                                          batch_size = batch_size,
                                                                          batchsize_for_val = batchsize_for_val,
                                                                          num_workers=5, 
                                                                          pin_memory=True)

In [9]:
# define model classes

## classic model
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2).squeeze()
    
# FGN model    
class FGN_M5(nn.Module):
    
    # changes:
    # nn.Conv1d -> fgnl.FGN_Conv1d
    # added g to conv inputs and outputs
    # make sure you pass g through the same pooling steps as x
    
    def __init__(self, n_input=1, n_output=35, stride=16, n_channel=32):
        super().__init__()
        self.fgn_conv1 = fgnl.FGN_Conv1d(in_channels=n_input, out_channels=n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.fgn_conv2 = fgnl.FGN_Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.fgn_conv3 = fgnl.FGN_Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.fgn_conv4 = fgnl.FGN_Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)
        
    def forward(self, x):
        x, g = self.fgn_conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        g = self.pool1(g)
        x, g = self.fgn_conv2(x, g)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        g = self.pool2(g)
        x, g = self.fgn_conv3(x ,g)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        g = self.pool3(g)
        x, _ = self.fgn_conv4(x, g)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=2).squeeze()

In [10]:
# pretrained models paths
save_path = '../Experiments/sample_SPEECHCOMMANDS_models/'

classic_model_name= 'sample_classic_model_SPEECHCOMMANDS'
fgn_model_name = 'sample_FGN_model_SPEECHCOMMANDS'

In [11]:
# define and load the models
# classic model
classic_model = M5()
classic_model.load_state_dict(torch.load(save_path+classic_model_name+'_state_dict.pth'))
classic_model.to(device)

# fgn model trained from scratch
fgn_model_from_scratch = FGN_M5()
fgn_model_from_scratch.load_state_dict(torch.load(save_path+fgn_model_name+'_state_dict.pth'))
fgn_model_from_scratch.to(device)

# converted fgn model (no retraining)
fgn_model_converted_no_retraining = FGN_M5()
fgn_model_converted_no_retraining.load_state_dict(torch.load(save_path+'sample_FGN_converted_model_SPEECHCOMMANDS'+'_state_dict.pth'))
fgn_model_converted_no_retraining.to(device)

# converted and retrained 1 epoch fgn model
fgn_model_converted_fast_retraining = FGN_M5()
fgn_model_converted_fast_retraining.load_state_dict(torch.load(save_path+'sample_FGN_converted_fast_retrained_model_SPEECHCOMMANDS'+'_state_dict.pth'))
fgn_model_converted_fast_retraining.to(device)

# converted and retrained 21 epoch fgn model
fgn_model_converted_long_retraining = FGN_M5()
fgn_model_converted_long_retraining.load_state_dict(torch.load(save_path+'sample_FGN_converted_long_retrained_model_SPEECHCOMMANDS'+'_state_dict.pth'))
fgn_model_converted_long_retraining.to(device)

FGN_M5(
  (fgn_conv1): FGN_Conv1d(
    (Conv1d): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  )
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv2): FGN_Conv1d(
    (Conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv3): FGN_Conv1d(
    (Conv1d): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  )
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv4): FGN_Conv1d(
    (Conv1d): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  )
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): Ma

In [12]:
# set all models to eval mode
classic_model.eval()
fgn_model_from_scratch.eval()
fgn_model_converted_no_retraining.eval()
fgn_model_converted_fast_retraining.eval()
fgn_model_converted_long_retraining.eval()

FGN_M5(
  (fgn_conv1): FGN_Conv1d(
    (Conv1d): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  )
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv2): FGN_Conv1d(
    (Conv1d): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  )
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv3): FGN_Conv1d(
    (Conv1d): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  )
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fgn_conv4): FGN_Conv1d(
    (Conv1d): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  )
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): Ma

In [13]:
# functions to test models
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def test(model, loader):
    model.eval()
    correct = 0
    for data, target in loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

    print(f'Accuracy: {correct}/{len(loader.dataset)} ({100. * correct / len(loader.dataset):.0f}%)')

In [14]:
# # verify accuracies
print('Train/Test/Val accuracy for Classic Model')
test(classic_model, train_loader)
test(classic_model, val_loader)
test(classic_model, test_loader)

print('Train/Test/Val accuracy for FGN model trained from scratch')
test(fgn_model_from_scratch, train_loader)
test(fgn_model_from_scratch, val_loader)
test(fgn_model_from_scratch, test_loader)

print('Train/Test/Val accuracy for FGN model converted from classic (no retraining)')
test(fgn_model_converted_no_retraining, train_loader)
test(fgn_model_converted_no_retraining, val_loader)
test(fgn_model_converted_no_retraining, test_loader)

print('Train/Test/Val accuracy for FGN model converted from classic (fast retraining)')
test(fgn_model_converted_fast_retraining, train_loader)
test(fgn_model_converted_fast_retraining, val_loader)
test(fgn_model_converted_fast_retraining, test_loader)

print('Train/Test/Val accuracy for FGN model converted from classic (long retraining)')
test(fgn_model_converted_long_retraining, train_loader)
test(fgn_model_converted_long_retraining, val_loader)
test(fgn_model_converted_long_retraining, test_loader)

Train/Test/Val accuracy for Classic Model
Accuracy: 76655/84843 (90%)
Accuracy: 8755/9981 (88%)
Accuracy: 9468/11005 (86%)
Train/Test/Val accuracy for FGN model trained from scratch
Accuracy: 73489/84843 (87%)
Accuracy: 8572/9981 (86%)
Accuracy: 9244/11005 (84%)
Train/Test/Val accuracy for FGN model converted from classic (no retraining)
Accuracy: 76655/84843 (90%)
Accuracy: 8755/9981 (88%)
Accuracy: 9468/11005 (86%)
Train/Test/Val accuracy for FGN model converted from classic (fast retraining)
Accuracy: 76264/84843 (90%)
Accuracy: 8654/9981 (87%)
Accuracy: 9394/11005 (85%)
Train/Test/Val accuracy for FGN model converted from classic (long retraining)
Accuracy: 77561/84843 (91%)
Accuracy: 8726/9981 (87%)
Accuracy: 9411/11005 (86%)


In [14]:
### Start Attacking the models

In [15]:
import foolbox

In [16]:
import numpy as np

In [17]:
# # extract values
# (waveforms, sample_rates, target_labels, speaker_ids, utterance_numbers) = (zip(*[train_loader.dataset[i] 
#                                                                                       for i in range(int(len(train_loader.dataset)/1))]))

In [18]:
# use this until I figure out the rest 
for waveforms, target_labels in val_loader:
    break
waveforms=waveforms.to(device)
target_labels=target_labels.to(device)

In [19]:
# # make sure all waveforms have the same length
### !! Normally padding is already done.
# waveforms = torch.stack([torch.nn.functional.pad(x, pad=(0, 16000 - x.numel()), mode='constant') for x in  waveforms])

In [20]:
labels = ['backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow', 'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine',
 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero']
print(len(labels))

35


In [21]:
# convert target labels to right format ('backward'->[1,0,0,0,...])
# this first line is not needed when running from val_load batch
# target_labels = torch.tensor([torch.tensor(labels.index(word)) for word in target_labels])

# target_labels = torch.tensor([np.random.randint(len(labels)) for _ in range(32)])

In [22]:
# check that all labels are in the targets
print([l in target_labels for l in range(len(labels))])

[False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]


In [23]:
# set model bounds and preprocessing

# precomputed bounds min and max input values
min_bound = -1.3844940662384033
max_bound = 1.3773366212844849
# min_bound = -10
# max_bound = 10

bounds = (min_bound, max_bound)
# preprocessing - I think these would be used in similar way to pytorch preprocessing
# but possible passed to whatever architecture is used (torch, tensorflow, other) 
# in my case the dataloaders already normalizes the data
preprocessing = dict(mean=0, std=1)

In [24]:
# ready the models for foolbox
classic_f_model = foolbox.PyTorchModel(classic_model, bounds=bounds,
                                       preprocessing=preprocessing, device=device)

fgn_f_model_from_scratch = foolbox.PyTorchModel(fgn_model_from_scratch, bounds=bounds,
                                       preprocessing=preprocessing, device=device)

fgn_f_model_converted_no_retraining = foolbox.PyTorchModel(fgn_model_converted_no_retraining, bounds=bounds,
                                       preprocessing=preprocessing, device=device)

fgn_f_model_converted_fast_retraining = foolbox.PyTorchModel(fgn_model_converted_fast_retraining, bounds=bounds,
                                       preprocessing=preprocessing, device=device)

fgn_f_model_converted_long_retraining = foolbox.PyTorchModel(fgn_model_converted_long_retraining, bounds=bounds,
                                       preprocessing=preprocessing, device=device)


In [25]:
# # check model accuracies ()
# print('accuracies for classic and fgn models')
# print(foolbox.accuracy(classic_f_model, waveforms, target_labels))
# print(foolbox.accuracy(fgn_f_model_from_scratch, waveforms, target_labels))
# print(foolbox.accuracy(fgn_f_model_converted_no_retraining, waveforms, target_labels))
# print(foolbox.accuracy(fgn_f_model_converted_fast_retraining, waveforms, target_labels))
# print(foolbox.accuracy(fgn_f_model_converted_long_retraining, waveforms, target_labels))

In [26]:
from tqdm.notebook import tqdm

In [27]:
# foolbox.accuracy doesn't work with dataloaders, so building a custom func to do so
def f_accuracy(model, dataloader, proc_func=None):
    # given a model and a dataloader, computes accuracy
    # proc_func is a processing function to apply to the values of the dataloader
    # that returns (inputs, targets)
    
    # get model device
    device = model.device
    
    running_count = 0
    running_average = 0
    # go through the dataset (assumes inputs and target are what is returned )
    for batch in tqdm(dataloader):
        # apply proc_func 
        if proc_func != None:
            inputs, targets = proc_func(*batch)
        else:
            inputs, targets = batch
        
        # send data to proper device
        inputs = inputs.to(device)
        targets = targets.to(device)

        # update running average accuracy and count
        running_average = (len(inputs)*foolbox.utils.accuracy(model, inputs, targets) + running_count*running_average)/(len(inputs)+running_count)
        running_count += len(inputs)
    
    return(running_average)


In [28]:
# check model accuracies
print('Train/Test/Val accuracy for Classic Model')
print(f_accuracy(classic_f_model, train_loader))
print(f_accuracy(classic_f_model, val_loader))
print(f_accuracy(classic_f_model, test_loader))

Train/Test/Val accuracy for Classic Model



0.9034923329007467



0.8771666166371758



0.860336210726608


In [29]:
print('Train/Test/Val accuracy for FGN model trained from scratch')
print(f_accuracy(fgn_f_model_from_scratch, train_loader))
print(f_accuracy(fgn_f_model_from_scratch, val_loader))
print(f_accuracy(fgn_f_model_from_scratch, test_loader))

Train/Test/Val accuracy for FGN model trained from scratch



0.8661763492638876



0.8588317804484171



0.8399818263558677


In [30]:
print('Train/Test/Val accuracy for FGN model converted from classic (no retraining)')
print(f_accuracy(fgn_f_model_converted_no_retraining, train_loader))
print(f_accuracy(fgn_f_model_converted_no_retraining, val_loader))
print(f_accuracy(fgn_f_model_converted_no_retraining, test_loader))

Train/Test/Val accuracy for FGN model converted from classic (no retraining)



0.9034923329007465



0.8771666166371758



0.860336210726608


In [31]:
print('Train/Test/Val accuracy for FGN model converted from classic (fast retraining)')
print(f_accuracy(fgn_f_model_converted_fast_retraining, train_loader))
print(f_accuracy(fgn_f_model_converted_fast_retraining, val_loader))
print(f_accuracy(fgn_f_model_converted_fast_retraining, test_loader))

Train/Test/Val accuracy for FGN model converted from classic (fast retraining)



0.8988838207076367



0.867047390106768



0.8536119945100198


In [32]:
print('Train/Test/Val accuracy for FGN model converted from classic (long retraining)')
print(f_accuracy(fgn_f_model_converted_long_retraining, train_loader))
print(f_accuracy(fgn_f_model_converted_long_retraining, val_loader))
print(f_accuracy(fgn_f_model_converted_long_retraining, test_loader))

Train/Test/Val accuracy for FGN model converted from classic (long retraining)



0.9141708803340147



0.8742610961482465



0.8551567468465542


In [28]:
### looks like they are the same, continue attacks

In [29]:
# attack params to explore
epsilons = torch.tensor([(max_bound-min_bound)*x 
            for x in 
            [0.0,
             1/256,
             3/512,
             1/128,
             3/256,
             1/64,
             3/128,
             1/32,
             3/64,
             1/16,
             3/32,
             1/8,
             3/16,
             1/4,
             3/8,
             1/2,
             3/4,
             1.0,] ], device=device)

print('epsilons: {}'.format(epsilons))

epsilons: tensor([ 0.0000,  0.0781,  0.1172,  0.1562,  0.2344,  0.3125,  0.4688,  0.6250,
         0.9375,  1.2500,  1.8750,  2.5000,  3.7500,  5.0000,  7.5000, 10.0000,
        15.0000, 20.0000], device='cuda:1')


In [30]:
### attack parameters

L2CarliniWagner_attack=foolbox.attacks.L2CarliniWagnerAttack()
# LinfPGD_attack=foolbox.attacks.LinfPGD()

# targetted vs untargetted
from foolbox.criteria import Misclassification

In [ ]:
L2CarliniWagner_attack()

In [31]:
# define the entire attack function using epsilons, criterion,

def L2CarliniWagner_attack_func(f_model, inputs, labels):
    device = f_model.device
    inputs = inputs.to(device)
    criterions = Misclassification(labels.to(device))
    return L2CarliniWagner_attack(model=f_model, inputs=inputs, criterion=criterions, epsilons=epsilons)

In [32]:
# write a function that attacks a model using a dataloader

def perform_attack(attack_func, f_model, dataloader):
    # iterates over dataloader 
    # attack is the attack function AFTER being defined: ex LinfPGD_attack=foolbox.attacks.LinfPGD()
    # (so it's the output foolbox.attacks.LinfPGD(), not foolbox.attacks.LinfPGD itself)
    # ensure the dataloader iterator returns (inputs, labels) 
    
    # defines results to return, shape is (epsilons, sample, (sample shape))=(18,32xbatches,1,8000)
    num_epsilons = 18 # hardcoded for now
    data_shape = (1,8000) # next(iter(dataloader))[0].shape[1:] # this could be expensive, hardcoded for now
    # create empty lists of the right shape
    results = {'adv_raw':np.array([]).reshape((num_epsilons, 0, *(data_shape))),
               'adv_clipped':np.array([]).reshape((num_epsilons, 0, *(data_shape))),
               'adv_success':np.array([]).reshape((num_epsilons, 0))}
    
    # iterate over loader
    for inputs, labels in tqdm(dataloader):
        
        # attack
        adv_raw, adv_clipped, adv_success = attack_func(f_model = f_model, 
                                                        inputs = inputs, 
                                                        labels =labels
                                                       )
        # compile with results
        results['adv_raw'] = np.concatenate([results['adv_raw'],
                                             np.array([x.cpu().numpy() for x in adv_raw])],
                                            axis=1)
        results['adv_clipped'] = np.concatenate([results['adv_clipped'],
                                                 np.array([x.cpu().numpy() for x in adv_clipped])],
                                                axis=1)
        results['adv_success'] = np.concatenate([results['adv_success'],
                                                 np.array([x.cpu().numpy() for x in adv_success])],
                                                axis=1)
    
    # return results dictionary
    return(results)

In [33]:
classic_results = perform_attack(L2CarliniWagner_attack_func, classic_f_model, val_loader)

In [51]:
import IPython.display as ipd

In [63]:
classic_results['adv_success'][-1][0]

1.0

In [65]:
waveform_first, *_ = classic_results['adv_clipped'][-1][1000]
ipd.Audio(waveform_first, rate=8000)

In [67]:
classic_results['adv_clipped'][0][1000]

array([[-4.33688692e-04, -8.05183896e-04,  7.93487561e-05, ...,
         3.53582669e-04,  2.24959935e-04,  4.96708322e-04]])

In [66]:
classic_results['adv_clipped'][-1][1000]

array([[-0.00055116, -0.00098984, -0.00014382, ...,  0.00035358,
         0.00022496,  0.00049671]])

In [36]:
classic_results['adv_raw'].shape

(18, 9981, 1, 8000)

In [40]:
import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType


def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

In [41]:
getsize(classic_results)

22997661904

In [46]:
# results dict is too large to save on its own apparently (about 22GB total), so split it up
import pickle as pickle
with open('../Experiments/adversarial_attacks_results/L2CarliniWagner_classic_results_adv_raw.pickle', 'wb') as handle:
    pickle.dump(classic_results['adv_raw'], handle, protocol=4)

In [47]:
with open('../Experiments/adversarial_attacks_results/L2CarliniWagner_classic_results_adv_clipped.pickle', 'wb') as handle:
    pickle.dump(classic_results['adv_clipped'], handle, protocol=4)

In [48]:
with open('../Experiments/adversarial_attacks_results/L2CarliniWagner_classic_results_adv_success.pickle', 'wb') as handle:
    pickle.dump(classic_results['adv_success'], handle, protocol=4)

In [125]:
# results are returned per epsilon, per sample
classic_results = {'classic_adv_raw':[], 'classic_adv_clipped':[], 'classic_adv_success':[]}
fgn_scratch_results = {'fgn_scratch_adv_raw':[], 'fgn_scratch_adv_clipped':[], 'fgn_scratch_adv_success':[]}
fgn_conv_results = {'fgn_conv_adv_raw':[], 'fgn_conv_adv_clipped':[], 'fgn_conv_adv_success':[]}
fgn_fast_results = {'fgn_fast_adv_raw':[], 'fgn_fast_adv_clipped':[], 'fgn_fast_adv_success':[]}
fgn_long_results = {'fgn_long_adv_raw':[], 'fgn_long_adv_clipped':[], 'fgn_long_adv_success':[]}

# attack the models over the train data
for waveforms, target_labels in tqdm(train_loader):
    waveforms = waveforms.to(device)
    criterion = Misclassification(target_labels.to(device))

    
    classic_adv_raw, classic_adv_clipped, classic_adv_success = L2CarliniWagner_attack(model=classic_f_model,                                                            
                                                                inputs=waveforms,
                                                                epsilons=epsilons,
                                                                criterion=criterion)
#     classic_results['classic_adv_raw'] = np.stack(classic_adv_raw.cpu().numpy(),classic_results['classic_adv_raw'], axis=-1)
#     classic_results['classic_adv_clipped']+=(classic_adv_clipped)
#     classic_results['classic_adv_success']+=(classic_adv_success)
    
    
#     fgn_scratch_adv_raw, fgn_scratch_adv_clipped, fgn_scratch_adv_success = LinfPGD_attack(model=fgn_f_model_from_scratch, 
#                                                                 inputs=waveforms,
#                                                                 epsilons=epsilons,
#                                                                 criterion=criterion)
#     fgn_scratch_results['fgn_scratch_adv_raw']+=(fgn_scratch_adv_raw)
#     fgn_scratch_results['fgn_scratch_adv_raw']+=(fgn_scratch_adv_raw)
#     fgn_scratch_results['fgn_scratch_adv_raw']+=(fgn_scratch_adv_raw)
    
#     fgn_conv_adv_raw, fgn_conv_adv_clipped, fgn_conv_adv_success = LinfPGD_attack(model=fgn_f_model_converted_no_retraining, 
#                                                                 inputs=waveforms,
#                                                                 epsilons=epsilons,
#                                                                 criterion=criterion)
#     fgn_conv_results['fgn_conv_adv_raw']+=(fgn_conv_adv_raw)
#     fgn_conv_results['fgn_conv_adv_clipped']+=(fgn_conv_adv_clipped)
#     fgn_conv_results['fgn_conv_adv_success']+=(fgn_conv_adv_success)
    
#     fgn_fast_adv_raw, fgn_fast_adv_clipped, fgn_fast_adv_success = LinfPGD_attack(model=fgn_f_model_converted_fast_retraining, 
#                                                                 inputs=waveforms,
#                                                                 epsilons=epsilons,
#                                                                 criterion=criterion)
#     fgn_fast_results['fgn_fast_adv_raw']+=(fgn_fast_adv_raw)
#     fgn_fast_results['fgn_fast_adv_clipped']+=(fgn_fast_adv_clipped)
#     fgn_fast_results['fgn_fast_adv_success']+=(fgn_fast_adv_success)
    
#     fgn_long_adv_raw, fgn_long_adv_clipped, fgn_long_adv_success = LinfPGD_attack(model=fgn_f_model_converted_long_retraining, 
#                                                                 inputs=waveforms,
#                                                                 epsilons=epsilons,
#                                                                 criterion=criterion)
#     fgn_long_results['fgn_long_adv_raw']+=(fgn_long_adv_raw)
#     fgn_long_results['fgn_long_adv_clipped']+=(fgn_long_adv_clipped)
#     fgn_long_results['fgn_long_adv_success']+=(fgn_long_adv_success)
    

KeyboardInterrupt: 

In [133]:
np.array([x.cpu().numpy() for x in classic_results['classic_adv_raw']]).shape

(0,)

In [162]:
classic_results['classic_adv_raw'] = np.empty((18,0,*(1,8000)))

In [163]:
np.concatenate([np.array([]).reshape((18,0,1,8000)),
                np.array([x.cpu().numpy() for x in classic_adv_raw])],
                axis=1
              ).shape

(18, 32, 1, 8000)

In [78]:
classic_adv_success.cpu().numpy().shape

(18, 32)

In [82]:
np.array([x.cpu().numpy() for x in classic_adv_raw]).shape

(18, 32, 1, 8000)

In [83]:
np.array([x.cpu().numpy() for x in classic_adv_clipped]).shape

(18, 32, 1, 8000)

In [165]:
np.concatenate([np.array([x.cpu().numpy() for x in classic_adv_success]),
         np.array([x.cpu().numpy() for x in classic_adv_success])],
         axis=1).shape


(18, 64)

In [57]:
# MAKE SURE TO SAVE RESULTS!
classic_results['classic_adv_success']

[tensor([False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False], device='cuda:1'),
 tensor([True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True], device='cuda:1'),
 tensor([True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True], device='cuda:1'),
 tensor([True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True], device='cuda:1'),
 tensor

In [50]:
classic_acc_per_eps = 1.0 - classic_results['adv_success'].mean(axis=-1)
print('Classic Accuracy per epsilon', classic_acc_per_eps)

Classic Accuracy per epsilon [8.77166617e-01 2.82035868e-01 2.02785292e-01 1.46879070e-01
 9.90882677e-02 7.96513375e-02 4.85923254e-02 3.77717664e-02
 3.00571085e-02 2.46468290e-02 1.93367398e-02 1.26239856e-02
 7.41408676e-03 3.80723374e-03 3.00571085e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [72]:
fgn_scratch_acc_per_eps = 1.0 - fgn_scratch_adv_success.cpu().numpy().mean(axis=-1)
print('FGN Scratch Accuracy per epsilon', fgn_scratch_acc_per_eps)

fgn_conv_acc_per_eps = 1.0 - fgn_conv_adv_success.cpu().numpy().mean(axis=-1)
print('FGN Converted Accuracy per epsilon', fgn_conv_acc_per_eps)

fgn_fast_acc_per_eps = 1.0 - fgn_fast_adv_success.cpu().numpy().mean(axis=-1)
print('FGN Fast Accuracy per epsilon', fgn_fast_acc_per_eps)

fgn_long_acc_per_eps = 1.0 - fgn_long_adv_success.cpu().numpy().mean(axis=-1)
print('FGN Long Accuracy per epsilon', fgn_long_acc_per_eps)

Classic Accuracy per epsilon [0.875 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.   ]
FGN Scratch Accuracy per epsilon [0.875 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.   ]
FGN Converted Accuracy per epsilon [0.875 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.   ]
FGN Fast Accuracy per epsilon [0.75 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.  ]
FGN Long Accuracy per epsilon [0.96875 0.      0.      0.      0.      0.      0.      0.      0.
 0.      0.      0.      0.      0.      0.      0.      0.      0.     ]


In [68]:
classic_adv_clipped[-1]

tensor([[[ 3.6183e-01,  7.3333e+00,  2.7586e+00,  ...,  9.0494e+00,
           9.4001e+00,  6.3108e+00]],

        [[ 1.0000e+01, -1.0000e+01, -7.6164e+00,  ...,  9.0764e+00,
           1.3882e+00, -1.0000e+01]],

        [[-1.0000e+01,  4.1035e+00,  1.0000e+01,  ..., -1.0000e+01,
           2.3013e+00, -1.0000e+01]],

        ...,

        [[ 4.9683e+00,  6.0000e+00, -1.1921e-07,  ..., -1.0000e+01,
           1.0000e+01,  1.0000e+01]],

        [[ 3.3333e+00,  1.3333e+00,  1.3333e+00,  ...,  1.0000e+01,
           6.1652e+00, -1.0000e+01]],

        [[-4.4840e+00,  3.3333e+00,  8.6667e+00,  ...,  1.0000e+01,
           1.0000e+01,  8.7990e-01]]], device='cuda:1')

In [51]:
# check model accuracies on adv attacks
print('accuracies for classic and fgn models')
print(foolbox.accuracy(classic_f_model, classic_adv_clipped[-1], target_labels))
# print(foolbox.accuracy(fgn_f_model_from_scratch, waveforms, target_labels))
# print(foolbox.accuracy(fgn_f_model_converted_no_retraining, waveforms, target_labels))
# print(foolbox.accuracy(fgn_f_model_converted_fast_retraining, waveforms, target_labels))
# print(foolbox.accuracy(fgn_f_model_converted_long_retraining, waveforms, target_labels))

accuracies for classic and fgn models


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cpu!

In [ ]:
import torch.nn.functional as F

In [ ]:
### build a dataloader

In [ ]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 1)

def collate_fn(batch, resample_rate):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets = [], []
    
    # resampling func
    transform = transforms.Resample(orig_freq=16000, new_freq=resample_rate)

    # Gather in lists, and encode labels as indices
    for waveform, _, label, *_ in batch:
        tensors += [transform(waveform)]
        targets += [label_to_index(label)]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets

In [ ]:
train_loader_sc = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(waveforms,target_labels),
        batch_size=32,
        shuffle=True,
        collate_fn=(lambda batch: collate_fn(batch, 8000)),
        num_workers=5,
        pin_memory=True,
    )